In [1]:
from lohrasb.best_estimator import BaseModel
from optuna.pruners import HyperbandPruner
from optuna.samplers._tpe.sampler import TPESampler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import optuna
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score
from interpret.glassbox import LogisticRegression
from interpret import show
from lohrasb.utils.metrics import f1_plus_tn



2023-02-21 21:04:56,520 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-02-21 21:04:56,525 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-02-21 21:04:56,527 :: matplotlib :: interactive is False
2023-02-21 21:04:56,528 :: matplotlib :: platform is darwin
2023-02-21 21:04:57,503 :: matplotlib :: CACHEDIR=/Users/hjavedani/.matplotlib
2023-02-21 21:04:57,506 :: matplotlib.font_manager :: Using fontManager instance from /Users/hjavedani/.matplotlib/fontlist-v330.json


#### Example 1 : Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

#### Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Define labels


In [3]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

#### Train test split

In [4]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]
y = y.values.ravel()


X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)

# for sample_weights
weights = np.ones(len(y_train))


#### Find feature types for later use

In [5]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#### Define model and set it argumens 

In [6]:
estimator = LogisticRegression()
estimator_params = {
        "penalty":["l2"],
        "C":[0.1,1],
        "max_iter":[500,700],

    }


In [7]:
obj = BaseModel().optimize_by_optuna(
            estimator=estimator,
            estimator_params=estimator_params,
            # f1_plus_tn is a custom metric 
            measure_of_accuracy="f1_plus_tn(y_true, y_pred)",
            fit_params = None,
            with_stratified=True,
            test_size=.3,
            verbose=3,
            n_jobs=-1,
            random_state=42,
            # optuna params
            # optuna study init params
            study=optuna.create_study(
                storage=None,
                sampler=TPESampler(),
                pruner=HyperbandPruner(),
                study_name="example of optuna optimizer",
                direction="maximize",
                load_if_exists=False,
                directions=None,
            ),
            # optuna optimization params
            study_optimize_objective=None,
            study_optimize_objective_n_trials=20,
            study_optimize_objective_timeout=600,
            study_optimize_n_jobs=-1,
            study_optimize_catch=(),
            study_optimize_callbacks=None,
            study_optimize_gc_after_trial=False,
            study_optimize_show_progress_bar=False,
        )

[I 2023-02-21 21:05:00,881] A new study created in memory with name: example of optuna optimizer


#### Build sklearn pipeline

In [8]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj),


 ])
 



#### Run Pipeline

In [9]:
pipeline.fit(X_train,y_train)
y_preds = pipeline.predict(X_test)
pred_labels = np.rint(y_preds)




/Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/optuna/study/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2023-02-21 21:05:02,674] Trial 1 finished with value: 15728.495737056715 and parameters: {'penalty': 'l2', 'C': 0.29809905290948, 'max_iter': 543}. Best is trial 1 with value: 15728.495737056715.
[I 2023-02-21 21:05:02,694] Trial 4 finished with value: 15728.495737056715 and parameters: {'penalty': 'l2', 'C': 0.38201587010084825, 'max_iter': 691}. Best is trial 1 with value: 15728.495737056715.
[I 2023-02-21 21:05:02,707] Trial 6 finished with value: 15729.495798319327 and parameters: {'penalty': 'l2', 'C': 0.11153266444582588, 'max_iter': 690}. Best is trial 6 with value: 15729.495798319327.
[I 2023-02-21 21:05:02,727] Trial 3 finished with value: 15728.495922905857 and parameters: {'penalty': 'l2', 'C': 0.5240

#### Check performance of the pipeline

In [10]:
print('F1 score : ')
print(f1_score(y_test,pred_labels))
print('Classification report : ')
print(classification_report(y_test,pred_labels))
print('Confusion matrix : ')
print(confusion_matrix(y_test,pred_labels))


F1 score : 
0.4938849856882644
Classification report : 
              precision    recall  f1-score   support

           0       0.83      0.96      0.89      8196
           1       0.73      0.37      0.49      2550

    accuracy                           0.82     10746
   macro avg       0.78      0.67      0.69     10746
weighted avg       0.81      0.82      0.80     10746

Confusion matrix : 
[[7852  344]
 [1601  949]]


#### Some estimators have predict_proba method as well

In [11]:
y_preds = pipeline.predict_proba(X_test)
print(y_preds)

[[0.92594476 0.07405524]
 [0.57011109 0.42988891]
 [0.47579328 0.52420672]
 ...
 [0.52781198 0.47218802]
 [0.73857442 0.26142558]
 [0.96454437 0.03545563]]


#### Part 2:  Use BestModel as a standalone estimator 

In [12]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


#### Transform features to make them ready for model input

In [13]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [14]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [15]:
obj.fit(X_train,y_train)
y_pred = obj.predict(X_test)

/Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/optuna/study/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2023-02-21 21:05:07,973] Trial 25 finished with value: 15728.495737056715 and parameters: {'penalty': 'l2', 'C': 0.8416819718282575, 'max_iter': 603}. Best is trial 7 with value: 15730.496045477013.
[I 2023-02-21 21:05:07,978] Trial 20 finished with value: 15728.495922905857 and parameters: {'penalty': 'l2', 'C': 0.8583422683695747, 'max_iter': 586}. Best is trial 7 with value: 15730.496045477013.
[I 2023-02-21 21:05:07,981] Trial 26 finished with value: 15728.495737056715 and parameters: {'penalty': 'l2', 'C': 0.87723328285712, 'max_iter': 591}. Best is trial 7 with value: 15730.496045477013.
[I 2023-02-21 21:05:08,001] Trial 22 finished with value: 15728.495737056715 and parameters: {'penalty': 'l2', 'C': 0.85

#### Check performance of the model

In [16]:

print('F1 score plus TN : ')
print(f1_plus_tn(y_test,pred_labels))
print('F1 score : ')
print(f1_score(y_test,pred_labels))
print('Classification report : ')
print(classification_report(y_test,pred_labels))
print('Confusion matrix : ')
print(confusion_matrix(y_test,pred_labels))

F1 score plus TN : 
7852.493884985688
F1 score : 
0.4938849856882644
Classification report : 
              precision    recall  f1-score   support

           0       0.83      0.96      0.89      8196
           1       0.73      0.37      0.49      2550

    accuracy                           0.82     10746
   macro avg       0.78      0.67      0.69     10746
weighted avg       0.81      0.82      0.80     10746

Confusion matrix : 
[[7852  344]
 [1601  949]]


In [17]:
obj.get_best_estimator()

In [18]:
obj.best_estimator

#### Get fitted randomized search object and its attributes

In [19]:
OptunaObj = obj.get_optimized_object()
OptunaObj.trials

[FrozenTrial(number=0, values=[15323.585312704294], datetime_start=datetime.datetime(2023, 2, 21, 21, 5, 1, 91545), datetime_complete=datetime.datetime(2023, 2, 21, 21, 5, 3, 273687), params={'penalty': 'l2', 'C': 0.4303715896801801, 'max_iter': 676}, distributions={'penalty': CategoricalDistribution(choices=('l2',)), 'C': UniformDistribution(high=1.0, low=0.1), 'max_iter': IntUniformDistribution(high=700, low=500, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=None),
 FrozenTrial(number=1, values=[15728.495737056715], datetime_start=datetime.datetime(2023, 2, 21, 21, 5, 1, 93549), datetime_complete=datetime.datetime(2023, 2, 21, 21, 5, 2, 674234), params={'penalty': 'l2', 'C': 0.29809905290948, 'max_iter': 543}, distributions={'penalty': CategoricalDistribution(choices=('l2',)), 'C': UniformDistribution(high=1.0, low=0.1), 'max_iter': IntUniformDistribution(high=700, low=500, step=1)}, user_attrs={}, system_attrs={}, inte

In [20]:
lr_global = obj.get_best_estimator().explain_global()
show(lr_global)


2023-02-21 21:05:11,801 :: interpret.provider.visualize :: Detected non-cloud environment.
2023-02-21 21:05:11,986 :: urllib3.connectionpool :: Starting new HTTP connection (1): 127.0.0.1:7001
2023-02-21 21:05:11,990 :: interpret.visual.udash :: Generating mini dash
2023-02-21 21:05:12,002 :: interpret.visual.udash :: Generated mini dash


<!-- http://127.0.0.1:7001/5384743056/ -->

In [21]:
lr_local = obj.get_best_estimator().explain_local(X_test[:50], y_test[:50])
show(lr_local)


2023-02-21 21:05:12,040 :: urllib3.connectionpool :: Starting new HTTP connection (1): 127.0.0.1:7001
2023-02-21 21:05:12,044 :: urllib3.connectionpool :: http://127.0.0.1:7001 "GET / HTTP/1.1" 200 2067
2023-02-21 21:05:12,046 :: interpret.visual.udash :: Generating mini dash
2023-02-21 21:05:12,066 :: interpret.visual.udash :: Generated mini dash


<!-- http://127.0.0.1:7001/5493858912/ -->

2023-02-21 21:05:20,186 :: interpret.visual.udash :: No overall plot to display: -1|LogisticRegression_1
